# 数据预处理与理解

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os, sys

sys.path.append("..")
from tools.common_used import crop_list
%matplotlib inline

## 原始数据的预处理

数据来源：

[Water footprint of both crop production (1961-2009) and consumption (1978-2009) in mainland China at the provincial level for 22 crops](https://waterfootprint.org/en/resources/waterstat/wf-crop-production-and-consumption-china/)

Downloads: [Spreadsheets](https://waterfootprint.org/media/downloads/WF_crop_production_and_consumption_China_at_provincial_level.zip) | [Paper](https://doi.org/10.1016/j.watres.2016.02.037)

Reference: Zhuo, L., Mekonnen, M.M. & Hoekstra, A.Y. (2016) The effect of inter-annual variability of consumption, production, trade and climate on crop-related green and blue water footprints and inter-regional virtual water trade: A study for China (1978-2008), Water Research, 94: 73–85

这个数据源有两套数据：
- Crop production：1961--2009，每个省的水足迹，通过省的作物生产量为基础数据制作的，没有消费量
- Crop consumption: 1978--2009，每个省地级市的水足迹消费量、国内进口量、国外进口量（Inner & International imports）

我们这里主要采用第二套数据，关注虚拟水足迹的省（地级市）之间的出口量。其实两者除了有没有贸易数据外，没有任何区别。

In [2]:
# 原始数据来源
source_path = r"../data/source/WF_crop_production_and_consumption_China_at_provincial_level/"

# 加载数据
test = pd.read_excel(r"{}/WF_crop_consumption_China_at_provincial_level_1978-2009.xlsx".format(source_path), header=1)

# 所有的标题
title = test.iloc[1:, :3]

# 我们这里关注 NVWI 数据
test.iloc[:, 8: 11]

,NVWI_Inter_Blue,NVWI_Inter_Green,NVWI_Inter_Total
0,-57.787101,-32.407631,-90.194733
1,28.909132,45.301547,74.210679
2,135.186161,213.299732,348.485893
3,95.944858,79.280064,175.224922
4,151.557236,137.672484,289.229720
...,...,...,...
987,-1418.946607,-801.497037,-2220.443644
988,-1288.144471,-513.245063,-1801.389534
989,-945.356963,-398.400435,-1343.757398
990,-1387.020494,-445.965617,-1832.986111


In [3]:
def get_table_by_colname(variable, file):
    """
    根据大的变量名，将所有该变量下的数据，分别存储
    param: variable, 需要保存的变量
    """
    result = title.copy()
    for i in range(len(crop_list)):
        crop = crop_list[i].title()
        tmp_data = pd.read_excel(file, sheet_name=f"{i+1}" + crop, header=1)
        result[crop] = tmp_data[variable]
    result.to_csv(f"../data/interim/{variable}.csv", index=False)
    return result


consumption = r"{}/WF_crop_consumption_China_at_provincial_level_1978-2009.xlsx".format(source_path)
production = r"{}/WF_crop_production_China_at_provincial_level_1961-2009.xlsx".format(source_path)
total = get_table_by_colname("NVWI_Inter_Total", consumption)
footprint = get_table_by_colname("WF_Total", production)

In [4]:
total.head()
footprint.head()

,Province Name,Province Code,Year,Wheat,Maize,Rice,Sorghum,Barley,Millet,Potato,...,Sugarbeet,Sugarcane,Cotton,Spinach,Tomato,Cabbage,Apple,Grape,Tea,Tobacco
1,Beijing,1,1979,74.210679,-202.548988,2167.611662,61.573556,25.898248,29.656194,60.822362,...,11.277791,31.530283,116.525242,-0.104435,5.044138,-0.115169,28.187313,-0.193126,36.908690,15.343575
2,Beijing,1,1980,348.485893,-469.240228,2008.891652,53.501047,32.405392,22.104911,45.497017,...,14.678364,29.944721,29.906058,-0.240176,4.356198,0.569861,29.046517,-0.241483,24.442461,11.350282
3,Beijing,1,1981,175.224922,-444.356016,2131.742579,59.675153,26.896964,17.714209,50.540135,...,15.156194,34.371399,129.736882,-0.482566,5.245837,0.518346,34.524341,-0.265388,27.687077,8.326199
4,Beijing,1,1982,289.229720,-561.901821,2040.696637,58.443023,24.713817,22.545120,58.807665,...,14.813250,42.501599,251.251187,-0.489691,5.228399,0.859771,35.401587,-0.332403,28.831569,19.395867
5,Beijing,1,1983,243.458895,-423.612109,2111.827123,48.704464,27.112838,28.133320,51.146220,...,18.288732,52.172649,376.820049,-0.385395,5.030235,0.818365,27.018137,-0.391425,29.650808,14.111434


,Province Name,Province Code,Year,Wheat,Maize,Rice,Sorghum,Barley,Millet,Potato,...,Sugarbeet,Sugarcane,Cotton,Spinach,Tomato,Cabbage,Apple,Grape,Tea,Tobacco
1,Beijing,1,1979,324.000000,595.704498,0.0,0.0,0.0,196.700000,4.181968,...,0.0,0.0,0.0,4.906000,0.0,10.820000,0.280000,0.728000,0.0,0.343293
2,Beijing,1,1980,237.001988,577.292573,0.0,0.0,0.0,210.171257,4.001708,...,0.0,0.0,0.0,3.333038,0.0,8.553046,0.306101,0.652284,0.0,0.412819
3,Beijing,1,1981,317.000000,590.641726,0.0,0.0,0.0,206.600000,3.514350,...,0.0,0.0,0.0,2.701000,0.0,6.253000,0.380000,0.657900,0.0,0.398832
4,Beijing,1,1982,400.018975,615.517766,0.0,0.0,0.0,203.056598,3.581807,...,0.0,0.0,0.0,3.316230,0.0,7.647706,0.326547,0.884355,0.0,0.449237
5,Beijing,1,1983,589.000000,774.065342,0.0,0.0,0.0,215.900000,2.916651,...,0.0,0.0,0.0,3.130000,0.0,6.380000,0.700000,0.883000,0.0,0.476408


## 测试一下数据并理解

**测试一下 2008 年的国内虚拟水贸易总额**

In [5]:
# 加载数据
total = pd.read_csv(r"../data/interim/NVWI_Inter_Total.csv")
total.head()

,Province Name,Province Code,Year,Wheat,Maize,Rice,Sorghum,Barley,Millet,Potato,...,Sugarbeet,Sugarcane,Cotton,Spinach,Tomato,Cabbage,Apple,Grape,Tea,Tobacco
0,Beijing,1,1979,74.210679,-202.548988,2167.611662,61.573556,25.898248,29.656194,60.822362,...,11.277791,31.530283,116.525242,-0.104435,5.044138,-0.115169,28.187313,-0.193126,36.908690,15.343575
1,Beijing,1,1980,348.485893,-469.240228,2008.891652,53.501047,32.405392,22.104911,45.497017,...,14.678364,29.944721,29.906058,-0.240176,4.356198,0.569861,29.046517,-0.241483,24.442461,11.350282
2,Beijing,1,1981,175.224922,-444.356016,2131.742579,59.675153,26.896964,17.714209,50.540135,...,15.156194,34.371399,129.736882,-0.482566,5.245837,0.518346,34.524341,-0.265388,27.687077,8.326199
3,Beijing,1,1982,289.229720,-561.901821,2040.696637,58.443023,24.713817,22.545120,58.807665,...,14.813250,42.501599,251.251187,-0.489691,5.228399,0.859771,35.401587,-0.332403,28.831569,19.395867
4,Beijing,1,1983,243.458895,-423.612109,2111.827123,48.704464,27.112838,28.133320,51.146220,...,18.288732,52.172649,376.820049,-0.385395,5.030235,0.818365,27.018137,-0.391425,29.650808,14.111434


**数据结构：**

省名-省代码-年份，22种作物的虚拟水交易量，NVWI = Net Virtual Water Import

- 如果 Import 是负数，说明是出口（水资源单位）
- 单位是 $M m^3$ 即百万立方米，和 billion 相比以 1000 为单位换算

In [6]:
# 以参考文章里提供的2008年值为依据，进行计算测试
total_2008 = total.groupby("Year").get_group(2008).iloc[:, 3:]

# 所有谷物的出口量
print("所有谷物的水资源出口量，和文章内的数字相同：")
total_2008[total_2008 < 0].abs().iloc[:, :6].sum().sum() / 1000

# 所有的作物
print("所有作物：")
total_2008[total_2008 < 0].abs().sum().sum()/1000

所有谷物的水资源出口量，和文章内的数字相同：


207.76623550902843

所有作物：


280.32846206048464

参考原始文章里用的 Virtual Water Flow 是所有谷物加起来的数值，我们这里要使用所有作物的值。

**进一步探索蓝水绿水**

In [7]:
# 加载数据
green = pd.read_csv(r"../data/interim/green_nvwi_inner.csv")
blue = pd.read_csv(r"../data/interim/blue_nvwi_inner.csv")

In [8]:
# 同理，分别分析蓝水和绿水
green_2008 = green.groupby("Year").get_group(2008).iloc[:, 3:]
blue_2008 = blue.groupby("Year").get_group(2008).iloc[:, 3:]

green_2008[green_2008 > 0].sum().sum() / 1000 + blue_2008[blue_2008 > 0].sum().sum() / 1000

279.48887265558506

这说明 green + blue 和 total 是一样的，总的水贸易量是蓝水和绿水的和

## Test for genepy index

In [9]:
import numpy as np
import sys

test = np.random.random((32, 22))
sys.path.append("..")

In [20]:
from module.genepy_index import genepy_index
from module.trade_table import TradeTable

TradeTable

array([[0.02655797],
       [0.02624486],
       [0.02550361],
       [0.02730018],
       [0.02797273],
       [0.02854386],
       [0.02888577],
       [0.02919412],
       [0.02788898],
       [0.02837765],
       [0.02855981],
       [0.02745664],
       [0.02748719],
       [0.02799482],
       [0.02704802],
       [0.02740221],
       [0.02585195],
       [0.02889715],
       [0.02821023],
       [0.02663999],
       [0.02667998],
       [0.02879994],
       [0.0279819 ],
       [0.0277389 ],
       [0.0279151 ],
       [0.02883925],
       [0.02629056],
       [0.0272956 ],
       [0.02817643],
       [0.02654431],
       [0.02765735],
       [0.02755575]])

array([[0.02655797],
       [0.02624486],
       [0.02550361],
       [0.02730018],
       [0.02797273],
       [0.02854386],
       [0.02888577],
       [0.02919412],
       [0.02788898],
       [0.02837765],
       [0.02855981],
       [0.02745664],
       [0.02748719],
       [0.02799482],
       [0.02704802],
       [0.02740221],
       [0.02585195],
       [0.02889715],
       [0.02821023],
       [0.02663999],
       [0.02667998],
       [0.02879994],
       [0.0279819 ],
       [0.0277389 ],
       [0.0279151 ],
       [0.02883925],
       [0.02629056],
       [0.0272956 ],
       [0.02817643],
       [0.02654431],
       [0.02765735],
       [0.02755575]])